In [1]:
!pip install 'markitdown[all]' ffmpeg-downloader ipykernel
!pip install llama-index llama-index-llms-openai_like llama-index-embeddings-openai llama-index-vector-stores-chroma iprogress ipywidgets jupyter chromadb==1.0.4 dotenv

In [2]:
import os
from markitdown import MarkItDown

result = None
if os.path.exists("./data/result.md") is False:
    md = MarkItDown(enable_plugins=False) # Set to True to enable plugins
    result = md.convert("./data/SENSIBLE_DATA.xlsx")
else:
    print("result.md already exists, skipping conversion.")

/home/vule/llama-rag/notebooks/.venv/lib/python3.12/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


result.md already exists, skipping conversion.


In [3]:
if result:
    with open("./data/result.md", "w", encoding="utf-8") as f:
        f.write(result.text_content)
else:
    print("File already exists.")

File already exists.


In [4]:
import os
from dotenv import load_dotenv

load_dotenv()

base_url = os.getenv("IONOS_BASE_URL", "http://localhost:11434")
api_key = os.getenv("IONOS_API_KEY", "your_api_key_here")

os.environ["OPENAI_API_BASE"] = base_url
os.environ["OPENAI_API_KEY"] = api_key

In [5]:
headers = {
    'Authorization': f'Bearer {api_key}',
    'Content-Type': 'application/json',
}

In [7]:
!pip install llama-index

In [8]:
from llama_index.llms.openai_like import OpenAILike

llama_3_3 = 'meta-llama/Llama-3.3-70B-Instruct'

llm = OpenAILike(
    api_base=base_url,
    temperature=0,
    model=llama_3_3,
    is_chat_model=True,
    default_headers=headers,
    api_key=api_key,
    context_window=128000,  # Adjusted to a more reasonable value for Llama 3.3-70B-Instruct
)

In [9]:
from llama_index.embeddings.openai import OpenAIEmbedding

embed_model_name = 'sentence-transformers/paraphrase-multilingual-mpnet-base-v2'

embed_model = OpenAIEmbedding(
  model_name=embed_model_name,
  api_base=base_url,
  api_key=api_key,
  default_headers=headers,
  embed_batch_size=10
)

In [10]:
from llama_index.core.settings import Settings

Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 1024 * 2

In [11]:
import time
from llama_index.core.agent.workflow import AgentStream
from IPython.display import display, Markdown

async def stream_and_time(handler):
    start = time.time()
    full_response_text = ""

    async for event in handler.stream_events():
        if isinstance(event, AgentStream):
            print(event.delta, end="", flush=True)
            full_response_text += event.delta

    end = time.time()
    
    execution_time = f"# Execution time: {end - start:.2f} seconds"
    display(Markdown(f"{execution_time}"))
    return full_response_text

In [13]:
import chromadb

chroma_client = chromadb.HttpClient()
chroma_collection = chroma_client.get_or_create_collection("use-case-C")

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given


In [37]:
import uuid

chat_id = str(uuid.uuid4())
chunks = chroma_collection.get(where={"chat_id": chat_id})

In [38]:
import nest_asyncio

from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.readers import SimpleDirectoryReader
from llama_index.core.indices import VectorStoreIndex
from llama_index.core import StorageContext
from llama_index.vector_stores.chroma import ChromaVectorStore

nest_asyncio.apply()

documents = SimpleDirectoryReader(input_files=['./data/result.md']).load_data()

chunks = chroma_collection.get(where={"chat_id": chat_id})
index = None
vector_store = ChromaVectorStore.from_collection(chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

if len(chunks['documents']) == 0:
    try:
        for doc in documents:
            doc.metadata = {
                "chat_id": chat_id,
            }

        
        transformations = [
            SentenceSplitter(
                chunk_size=1024 * 2,
                chunk_overlap=20,
            ),
        ]
        index = VectorStoreIndex.from_documents(documents, transformations=transformations, storage_context=storage_context, show_progress=True, insert_batch_size=512)
    except Exception as e:
        print(e)
else:
    index = VectorStoreIndex.from_vector_store(vector_store=vector_store, embed_model=embed_model, show_progress=True)
    print("Index already exists, skipping creation and loading existing index.")

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/512 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/512 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/512 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/512 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/512 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/512 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/435 [00:00<?, ?it/s]

In [51]:
from llama_index.core.vector_stores import (
    MetadataFilter,
    MetadataFilters,
    FilterOperator,
)

chunks = chroma_collection.get(where={"chat_id": chat_id})
if len(chunks['documents']) == 0:
    print("No documents found for the given chat_id.")
else:
    print(f"Found {len(chunks['documents'])} documents for chat_id: {chat_id}")
    
filters = MetadataFilters(
    filters=[
        MetadataFilter(key="chat_id", value=chat_id, operator=FilterOperator.EQ)
    ]
)

query_engine = index.as_query_engine(
    llm=llm,
    similarity_top_k=15,
    show_progress=True,
    filters=filters
)

tools = [
    QueryEngineTool(
      query_engine=query_engine,
        metadata=ToolMetadata(
            name="QueryAbstractionTool",
            description="A tool that is useful when you want to query through the documents and tries to analyze and model the documents for the stakeholders."
        )
    ),
]

Found 3507 documents for chat_id: 4b1251ee-1199-4911-82ac-e62c99cf4505


In [53]:
system_prompt = """ \n
You are a smart assistant designed to analyze complex insurance-related product data from an Excel file. 
You support the user by understanding data structures, modeling them, and translating them into suitable formats for stakeholders like domain experts, database admins, or frontend developers.

The Excel file includes structured extracts of insurance product configurations. 
Each contract component (e.g. savings part, risk module, or additional coverage) is modeled as a separate entity.
These groups are represented as attribute bundles with validity constraints (temporal and/or logical).

Your job is to reason step-by-step through user queries, potentially using tools in a chain of thought manner to:
- Understand the schema
- Identify entities and relationships
- Recommend modeling approaches (relational, NoSQL, frontend structure, etc.)
- Rephrase for different target audiences

## Tools

You have access to a set of specialized tools that help you analyze, 
extract, and process information effectively.
Use them wisely — not everything needs a tool, but they can help with complex or data-heavy tasks.

When a request is made, ask yourself:
- What do I need to figure out?
- Can I reason through it myself, or do I need to use a tool to get the answer?

If it makes sense to use a tool, break the task down clearly.
Choose the most suitable tool and provide it with clean, focused input. 
Once you get the result, interpret it and decide if anything else is needed.

## Output Format
Please answer in the same language as the user's input.
Think out loud before taking any action. This helps others understand your reasoning.

Repeat the thought → action → observation loop until you have enough to respond.

### When using a tool, follow this format:
Thought: [What you’re thinking and why you need the tool]
Action: [Tool name] (choose from {tool_names})
Action Input: [Tool input in JSON]
Observation: [Result you got from the tool]

### When you're done:
Thought: I have everything I need now.
Answer: [Your final answer here]

If you cannot answer:
Thought: I cannot answer the question with the provided tools.
Answer: [your answer here – same language as user]
"""
print(system_prompt)

 

You are a smart assistant designed to analyze complex insurance-related product data from an Excel file. 
You support the user by understanding data structures, modeling them, and translating them into suitable formats for stakeholders like domain experts, database admins, or frontend developers.

The Excel file includes structured extracts of insurance product configurations. 
Each contract component (e.g. savings part, risk module, or additional coverage) is modeled as a separate entity.
These groups are represented as attribute bundles with validity constraints (temporal and/or logical).

Your job is to reason step-by-step through user queries, potentially using tools in a chain of thought manner to:
- Understand the schema
- Identify entities and relationships
- Recommend modeling approaches (relational, NoSQL, frontend structure, etc.)
- Rephrase for different target audiences

## Tools

You have access to a set of specialized tools that help you analyze, 
extract, and process 

In [58]:
from llama_index.core.agent.workflow import ReActAgent
from llama_index.core.workflow import Context
from llama_index.core.memory import ChatMemoryBuffer

import nest_asyncio

nest_asyncio.apply()

chat_memory = ChatMemoryBuffer.from_defaults(
    token_limit=128.000,
    llm=llm,
)

agent = ReActAgent(
    tools=tools,
    llm=llm,
    system_prompt=system_prompt,
    max_iterations=3,
    max_execution_time=60,
    verbose=True,
)
chat_memory.reset()

ctx = Context(agent)

In [59]:
query = "Was sind das für Daten? Kannst du mir etwas modellieren?"
handler = agent.run(query, ctx=ctx, memory=chat_memory)
full_text_response = await stream_and_time(handler)

Running step init_run
Step init_run produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Thought: The current language of the user is: German. IThought: The current language of the user is: German. I need to use a need to use a tool to tool to help me answer the question.
Action: QueryAbstractionTool
Action Input: help me answer the question.
Action: QueryAbstractionTool
Action Input: {"input {"input": "": "Daten modellierenDaten modellieren"}"}Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced no event
Running step call_tool
Step run_agent_step produced event AgentOutput
Running step parse_agent_output
Step parse_agent_output produced no event
Running step call_tool
Step call_tool produced event ToolCallResult
Running step aggregate_tool_results
Step aggregate_tool_results produced event AgentInput
Running step setup_agent
Step setup_agent produced event Ag

# Execution time: 22.61 seconds

In [60]:
%%time

import pandas as pd
from IPython.display import display

pd.set_option('display.max_colwidth', None)

# Split the response into thought and answer
thought = ""
answer = ""
if "Thought:" in full_text_response and "Answer:" in full_text_response:
    parts = full_text_response.split("Answer:", 1)
    if len(parts) > 1:
        thought = parts[0].replace("Thought:", "").strip()
        answer = parts[1].strip()

# Create DataFrame with all columns
data = {
    "Query": [query],
    "Full Response": [full_text_response],
    "Thought": [thought],
    "Answer": [answer]
}
df = pd.DataFrame(data).T

# Formatting response
display(df.style.set_properties(**{
    'white-space': 'pre-wrap',
    'text-align': 'left',
    'font-family': 'monospace',
}))

,0
Query,Was sind das für Daten? Kannst du mir etwas modellieren?
Full Response,"Thought: The current language of the user is: German. I need to use a tool to help me answer the question. Action: QueryAbstractionTool Action Input: {""input"": ""Daten modellieren""}Thought: Ich kann jetzt antworten, ohne weitere Tools zu verwenden. Ich werde die Antwort in der gleichen Sprache wie die Frage des Benutzers geben. Answer: Das Datenmodellieren ist ein wichtiger Schritt bei der Erstellung von Versicherungsanwendungen. Durch die Identifizierung von Entitäten, Attributen und Beziehungen zwischen ihnen kann ein konsistentes und effizientes Datenmodell erstellt werden. Dieses Modell kann dann als Grundlage für die Implementierung von Datenbanken, Datenwarehouses oder anderen Daten-speicherungslösungen dienen. Ein Beispiel für ein solches Datenmodell könnte die oben genannten Komponenten umfassen, wie Entitäten, Attribute und Beziehungen. Durch die Verwendung eines solchen Datenmodells kann die Datenverwaltung effizienter und effektiver gestaltet werden."
Thought,"The current language of the user is: German. I need to use a tool to help me answer the question. Action: QueryAbstractionTool Action Input: {""input"": ""Daten modellieren""} Ich kann jetzt antworten, ohne weitere Tools zu verwenden. Ich werde die Antwort in der gleichen Sprache wie die Frage des Benutzers geben."
Answer,"Das Datenmodellieren ist ein wichtiger Schritt bei der Erstellung von Versicherungsanwendungen. Durch die Identifizierung von Entitäten, Attributen und Beziehungen zwischen ihnen kann ein konsistentes und effizientes Datenmodell erstellt werden. Dieses Modell kann dann als Grundlage für die Implementierung von Datenbanken, Datenwarehouses oder anderen Daten-speicherungslösungen dienen. Ein Beispiel für ein solches Datenmodell könnte die oben genannten Komponenten umfassen, wie Entitäten, Attribute und Beziehungen. Durch die Verwendung eines solchen Datenmodells kann die Datenverwaltung effizienter und effektiver gestaltet werden."


CPU times: user 12.7 ms, sys: 2.46 ms, total: 15.2 ms
Wall time: 14.2 ms


In [61]:
query = "Zeig mir eine ERD-Skizze dazu oder gib mir Tabellenstruktur-Vorschläge (Name, Spalten, Relationen)."
handler = agent.run(query, ctx=ctx, memory=chat_memory)
full_text_response = await stream_and_time(handler)

Running step init_run
Step init_run produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step init_run produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Thought: Der Benutzer benötThought: Der Benutzer benötigtigt eine ERD-S eine ERD-Skizze oder Tabellenkizze oder Tabellenstruktur-Vorschlägestruktur-Vorschläge. Ich muss zuer. Ich muss zuerst denst den Kontext verstehen, um eine Kontext verstehen, um eine genaue Antwort zu geben genaue Antwort zu geben. Leider habe. Leider habe ich nicht genug Informationen, um direkt ich nicht genug Informationen, um direkt eine eine ERD-Skizze ERD-Skizze oder Tab oder Tabellenstrellenstruktur-Vuktur-Vorschläge zu erstellenorschläge zu erstellen. Ich. Ich werde versuchen, den Ben werde versuchen, den Benutzerutzer zu unterstützen, indem ich zu unterstützen, indem ich ihn a ihn auffordere, mehruffordere, mehr Konte

# Execution time: 84.76 seconds

In [62]:
%%time

import pandas as pd
from IPython.display import display

pd.set_option('display.max_colwidth', None)

# Split the response into thought and answer
thought = ""
answer = ""
if "Thought:" in full_text_response and "Answer:" in full_text_response:
    parts = full_text_response.split("Answer:", 1)
    if len(parts) > 1:
        thought = parts[0].replace("Thought:", "").strip()
        answer = parts[1].strip()

# Create DataFrame with all columns
data = {
    "Query": [query],
    "Full Response": [full_text_response],
    "Thought": [thought],
    "Answer": [answer]
}
df = pd.DataFrame(data).T

# Formatting response
display(df.style.set_properties(**{
    'white-space': 'pre-wrap',
    'text-align': 'left',
    'font-family': 'monospace',
}))

CPU times: user 16.3 ms, sys: 196 μs, total: 16.5 ms
Wall time: 15.4 ms


In [63]:
query = "Kannst du mir eine Vorlage machen, wie ich die Tabellen gestalten kann? Mit Tabellenname und ihre Spalten bitte! Diese sind besonders wichtig beim Systementwurf für die Entwickler::innen, die das System umsetzen."
handler = agent.run(query, ctx=ctx, memory=chat_memory)
full_text_response = await stream_and_time(handler)

Running step init_run
Step init_run produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Step init_run produced event AgentInput
Running step setup_agent
Step setup_agent produced event AgentSetup
Running step run_agent_step
Thought: Die aktuelle Sprache des BenThought: Die aktuelle Sprache des Benutzers ist Deutsch. Ich muss einutzers ist Deutsch. Ich muss ein Tool verwenden, um eine passende Antwort zu Tool verwenden, um eine passende Antwort zu geben, aber ich denke, ich kann diese Frage geben, aber ich denke, ich kann diese Frage ohne Tool beantworten.

Die Gestaltung von ohne Tool beantworten.

Die Gestaltung von Tabellen ist ein wichtiger Aspekt Tabellen ist ein wichtiger Aspekt beim beim Systementwurf, da sie die Struktur und Systementwurf, da sie die Struktur und Organisation Organisation der Daten im System darstellt. der Daten im System darstellt. Hier ist eine allgemeine Vorlage, wie Hier ist eine allgemein

# Execution time: 10.31 seconds

In [64]:
%%time

import pandas as pd
from IPython.display import display

pd.set_option('display.max_colwidth', None)

# Split the response into thought and answer
thought = ""
answer = ""
if "Thought:" in full_text_response and "Answer:" in full_text_response:
    parts = full_text_response.split("Answer:", 1)
    if len(parts) > 1:
        thought = parts[0].replace("Thought:", "").strip()
        answer = parts[1].strip()

# Create DataFrame with all columns
data = {
    "Query": [query],
    "Full Response": [full_text_response],
    "Thought": [thought],
    "Answer": [answer]
}
df = pd.DataFrame(data).T

# Formatting response
display(df.style.set_properties(**{
    'white-space': 'pre-wrap',
    'text-align': 'left',
    'font-family': 'monospace',
}))

,0
Query,"Kannst du mir eine Vorlage machen, wie ich die Tabellen gestalten kann? Mit Tabellenname und ihre Spalten bitte! Diese sind besonders wichtig beim Systementwurf für die Entwickler::innen, die das System umsetzen."
Full Response,"Thought: Die aktuelle Sprache des Benutzers ist Deutsch. Ich muss ein Tool verwenden, um eine passende Antwort zu geben, aber ich denke, ich kann diese Frage ohne Tool beantworten. Die Gestaltung von Tabellen ist ein wichtiger Aspekt beim Systementwurf, da sie die Struktur und Organisation der Daten im System darstellt. Hier ist eine allgemeine Vorlage, wie man Tabellen gestalten kann: 1. **Benutzer-Tabelle** - Spalten: - Benutzer-ID (einzigartige Identifikationsnummer) - Benutzername - E-Mail-Adresse - Passwort (verschlüsselt) - Rolle (z.B. Administrator, Benutzer) 2. **Produkt-Tabelle** (falls zutreffend) - Spalten: - Produkt-ID - Produktname - Beschreibung - Preis - Verfügbarkeit 3. **Bestellung-Tabelle** (falls zutreffend) - Spalten: - Bestellungs-ID - Benutzer-ID (Foreign Key zur Benutzer-Tabelle) - Produkt-ID (Foreign Key zur Produkt-Tabelle) - Bestelldatum - Status (z.B. ""in Bearbeitung"", ""versandt"", ""storniert"") 4. **Kontakt-Tabelle** (falls zutreffend) - Spalten: - Kontakt-ID - Name - E-Mail-Adresse - Nachricht - Datum 5. **Log-Tabelle** (für Protokollierung von Systemereignissen) - Spalten: - Log-ID - Ereignis-Typ (z.B. Anmeldung, Abmeldung, Fehler) - Benutzer-ID (Foreign Key zur Benutzer-Tabelle) - Datum und Uhrzeit - Details zum Ereignis Diese Vorlage kann je nach den spezifischen Anforderungen des Systems angepasst und erweitert werden. Es ist wichtig, dass die Tabellennamen und Spaltennamen eindeutig und beschreibend sind, um die Datenmodellierung und den Systementwurf zu erleichtern. Answer: Ich hoffe, diese Vorlage hilft dir bei der Gestaltung deiner Tabellen. Wenn du spezifischere Anforderungen hast oder weitere Fragen zu bestimmten Aspekten des Systementwurfs hast, stehe ich gerne zur Verfügung."
Thought,"Die aktuelle Sprache des Benutzers ist Deutsch. Ich muss ein Tool verwenden, um eine passende Antwort zu geben, aber ich denke, ich kann diese Frage ohne Tool beantworten. Die Gestaltung von Tabellen ist ein wichtiger Aspekt beim Systementwurf, da sie die Struktur und Organisation der Daten im System darstellt. Hier ist eine allgemeine Vorlage, wie man Tabellen gestalten kann: 1. **Benutzer-Tabelle** - Spalten: - Benutzer-ID (einzigartige Identifikationsnummer) - Benutzername - E-Mail-Adresse - Passwort (verschlüsselt) - Rolle (z.B. Administrator, Benutzer) 2. **Produkt-Tabelle** (falls zutreffend) - Spalten: - Produkt-ID - Produktname - Beschreibung - Preis - Verfügbarkeit 3. **Bestellung-Tabelle** (falls zutreffend) - Spalten: - Bestellungs-ID - Benutzer-ID (Foreign Key zur Benutzer-Tabelle) - Produkt-ID (Foreign Key zur Produkt-Tabelle) - Bestelldatum - Status (z.B. ""in Bearbeitung"", ""versandt"", ""storniert"") 4. **Kontakt-Tabelle** (falls zutreffend) - Spalten: - Kontakt-ID - Name - E-Mail-Adresse - Nachricht - Datum 5. **Log-Tabelle** (für Protokollierung von Systemereignissen) - Spalten: - Log-ID - Ereignis-Typ (z.B. Anmeldung, Abmeldung, Fehler) - Benutzer-ID (Foreign Key zur Benutzer-Tabelle) - Datum und Uhrzeit - Details zum Ereignis Diese Vorlage kann je nach den spezifischen Anforderungen des Systems angepasst und erweitert werden. Es ist wichtig, dass die Tabellennamen und Spaltennamen eindeutig und beschreibend sind, um die Datenmodellierung und den Systementwurf zu erleichtern."
Answer,"Ich hoffe, diese Vorlage hilft dir bei der Gestaltung deiner Tabellen. Wenn du spezifischere Anforderungen hast oder weitere Fragen zu bestimmten Aspekten des Systementwurfs hast, stehe ich gerne zur Verfügung."


CPU times: user 4.97 ms, sys: 3.4 ms, total: 8.37 ms
Wall time: 7.49 ms
